In [1]:
import pandas as pd

from functions import train_test, RMSE, MAE

In [2]:
def naive_bayes(ratings, movies, genre_list):
    genre_length = len(genre_list)
    
    scores = [i/2 for i in range(1,11)]
    user_rating_count = {}

    for index, row in ratings.iterrows():
        score_counts = []

        for score in scores:
            score_counts.append(len(row[row==score]))

        user_rating_count[index] = score_counts
    
    rating_count_df = pd.DataFrame(user_rating_count, index=scores)
    prior_prob = ((rating_count_df+genre_length)/rating_count_df.sum()).T
    
    genre_index = {}
    for genre in genre_list:
        genre_index[genre] = list(movies[movies['genres'].str.contains(genre)]['movieId'].astype('str'))
    
    cond_prob = {}

    for index, row in ratings.iterrows():
        cond_df = pd.DataFrame([], index=scores)
    
        for genre in genre_list:
            sub_row = row[genre_index[genre]]
            score_count = []

            for s in scores:
                score_count.append(len(sub_row[sub_row == s]))

            cond_df[genre] = score_count
        
        cond_prob[index] = ((cond_df + 1).T/(rating_count_df[index]+genre_length)).T
            
            
    return prior_prob, cond_prob

In [3]:
def naive_bayes_predict(prior_prob, cond_prob, movies, genre_list, values):
    scores = [i/2 for i in range(1,11)]
    
    prediction_list = []
    
    for u,i in values:
        prediction = scores[0]
        best_prob = 0
        movie_genre = movies[movies['movieId'] == int(i)]['genres'].iloc[0].split('|')
        
        for rating in scores:
            current_prob = prior_prob.loc[u, rating]
            
            for genre in movie_genre:
                if genre in genre_list:
                    current_prob *= cond_prob[u].loc[rating, genre]
             
            if current_prob > best_prob:
                best_prob = current_prob
                
                prediction = rating
        
        prediction_list.append(prediction)
    
    return prediction_list

In [4]:
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('data/user_rating_pt.csv')

columns = ratings.columns
movies=movies[movies['movieId'].isin(columns)]

In [5]:
known = []

columns = list(ratings.columns)
index = list(ratings.index)

for u in index:
    for i in columns:
        if ratings.loc[u,i] > 0:
            known.append((u, i))

In [6]:
training, testing = train_test(known)

In [7]:
ratings_training = ratings.copy()

for u,i in testing:
    ratings_training.loc[u,i] = 0

In [8]:
genre_list = []

for index, row in movies.iterrows():
    genres = row['genres'].split('|')
    
    for g in genres:
        if g not in genre_list:
            genre_list.append(g)

In [9]:
prior_prob, cond_prob = naive_bayes(ratings_training, movies, genre_list)

/Users/fawadali/anaconda3/lib/python3.8/site-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [10]:
rating_pred = naive_bayes_predict(prior_prob, cond_prob, movies, genre_list, testing)

In [11]:
test_ratings = []
for u,i in testing:
    test_ratings.append(ratings.loc[u,i])

In [14]:
error = 0
for ind in range(len(rating_pred)):
    error += (rating_pred[ind] - test_ratings[ind])**2

(error/len(rating_pred))**.5

1.0694995715743782